In [28]:
import os
import zipfile
import shutil
import random
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from math import atan2, degrees

### 데이터 준비

In [2]:
# 초기화 후 CP949로 해제
!rm -rf /content/data && mkdir -p /content/data
!unzip -qq -O CP949 "/content/drive/MyDrive/Colab Notebooks/Python_colab/Web Service/생성형 AI/data/medicine.zip" -d /content/data

In [3]:
random.seed(2025)

IMG_SRC = Path("/content/data/원천데이터/TS1/result/medicine/images")
ANN_SRC = Path("/content/data/라벨링데이터/TL1/result/medicine/annotations")

# TS1 / TL1 바로 아래에 생성
IMG_DST_ROOT = Path("/content/data/원천데이터/TS1")
ANN_DST_ROOT = Path("/content/data/라벨링데이터/TL1")

for p in [IMG_DST_ROOT, ANN_DST_ROOT]:
    for s in ["train", "val", "test"]:
        (p/s).mkdir(parents=True, exist_ok=True)

# 유틸
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
def list_files(root, exts=None):
    exts = {e.lower() for e in (exts or [])}
    files = []
    for f in root.rglob("*"):
        if f.is_file() and (not exts or f.suffix.lower() in exts):
            files.append(f)
    return files

# 1) 분배 대상 이미지 수집 (이미 분배된 것은 제외)
split_dirs = {IMG_DST_ROOT/"train", IMG_DST_ROOT/"val", IMG_DST_ROOT/"test"}
all_imgs = [p for p in list_files(IMG_SRC, IMG_EXTS) if not any(str(p).startswith(str(sd)) for sd in split_dirs)]

# 2) 섞고 8/1/1 분할
random.shuffle(all_imgs)
n = len(all_imgs)
tr, va = int(n*0.8), int(n*0.9)
splits = {
    "train": all_imgs[:tr],
    "val":   all_imgs[tr:va],
    "test":  all_imgs[va:]
}

# 3) 이미지 이동 + 주석(어노테이션) 동반 이동
def find_annotation(img_path: Path):
    # 이미지 파일명(stem)과 같은 이름의 주석을 annotations 트리에서 탐색
    stem = img_path.stem
    # 주석 확장자 후보 (필요시 추가)
    cand_exts = [".json", ".xml", ".txt"]
    for ext in cand_exts:
        hits = list(ANN_SRC.rglob(stem + ext))
        if hits:
            return hits[0]
    return None

moved = {"train":0, "val":0, "test":0}
moved_ann = {"train":0, "val":0, "test":0}

for split, files in splits.items():
    for img in files:
        dst_img = IMG_DST_ROOT/split/img.name
        if not dst_img.exists():  # 재실행 시 중복 방지
            dst_img.parent.mkdir(parents=True, exist_ok=True)
            shutil.move(str(img), str(dst_img))
            moved[split] += 1

        ann = find_annotation(img)
        if ann:
            dst_ann = ANN_DST_ROOT/split/ann.name
            if not dst_ann.exists():  # 재실행 시 중복 방지
                dst_ann.parent.mkdir(parents=True, exist_ok=True)
                shutil.move(str(ann), str(dst_ann))
                moved_ann[split] += 1

print(f"[이미지 이동] train/val/test = {moved['train']}/{moved['val']}/{moved['test']}")
print(f"[주석 이동]  train/val/test = {moved_ann['train']}/{moved_ann['val']}/{moved_ann['test']}")

# 4) 요약
def count_in(dst_root):
    return {s: len(list((dst_root/s).glob("*"))) for s in ["train","val","test"]}

print("\n== 최종 파일 개수 ==")
print("이미지:", count_in(IMG_DST_ROOT))
print("주석 :", count_in(ANN_DST_ROOT))


[이미지 이동] train/val/test = 388/48/49
[주석 이동]  train/val/test = 388/48/49

== 최종 파일 개수 ==
이미지: {'train': 388, 'val': 48, 'test': 49}
주석 : {'train': 388, 'val': 48, 'test': 49}


### 환경세팅

In [4]:
# pip 최신화
!pip install -q --upgrade pip

In [5]:
# 1) PaddlePaddle (GPU, Colab T4 기준)
!pip install -q paddlepaddle-gpu==2.5.2.post117 \
    -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

# 2) OpenCV - paddleocr 2.7.0.3과 호환되는 버전
!pip uninstall -y opencv-python opencv-contrib-python >/dev/null
!pip install -q "opencv-python<=4.6.0.66" "opencv-contrib-python<=4.6.0.66"

# 3) PaddleOCR 본체 (자동 의존성 설치 막음)
!pip install -q --no-deps paddleocr==2.7.0.3

# 4) 이미지 OCR에 필요한 최소 의존성만 수동 설치
!pip install -q attrdict fire==0.5.0 lmdb shapely pyclipper \
    rapidfuzz scikit-image imgaug tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddleocr 2.7.0.3 requires pdf2docx, which is not installed.
paddleocr 2.7.0.3 requires premailer, which is not installed.
paddleocr 2.7.0.3 requires PyMuPDF<1.21.0, which is not installed.
paddleocr 2.7.0.3 requires python-docx, which is not installed.
paddleocr 2.7.0.3 requires visualdl, which is not installed.


In [6]:
!pip install -q "numpy<2.0"

### 전처리

In [7]:
# 1. 안전한 이미지 읽기
def safe_imread(path):
    path = str(path)
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

# 2. 안전한 이미지 저장
def safe_imsave(path, img):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    ext = path.suffix.lower() or ".jpg"
    ok, buf = cv2.imencode(ext, img)
    if ok:
        buf.tofile(str(path))

# 3. 리사이즈 (긴 변 기준)
def normalize_size(img, max_long=1280):
    h, w = img.shape[:2]
    long_side = max(h, w)
    if long_side <= max_long:
        return img
    scale = max_long / long_side
    new_w, new_h = int(w * scale), int(h * scale)
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

# 4. 그레이스케일 변환
def to_gray(img):
    if len(img.shape) == 2:
        return img
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 5. 가우시안 블러
def denoise_gaussian(img, ksize=3):
    k = max(3, int(ksize) | 1)  # 홀수 보정
    return cv2.GaussianBlur(img, (k, k), 0)

# 6. CLAHE + 오츠 이진화
def enhance_contrast_and_binarize(img, clip_limit=2.0, tile_grid_size=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    enhanced = clahe.apply(img)
    _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

In [8]:
def preprocess_image_pipeline(input_path, output_path,
                               max_long=1280,
                               blur_ksize=3,
                               clahe_clip=2.0,
                               clahe_tile=(8,8)):
    # 1. 로드
    img = safe_imread(input_path)
    if img is None:
        raise FileNotFoundError(f"이미지를 불러올 수 없음: {input_path}")

    # 2. 리사이즈
    img = normalize_size(img, max_long=max_long)

    # 3. 그레이스케일
    gray = to_gray(img)

    # 4. 가우시안 블러
    blurred = denoise_gaussian(gray, ksize=blur_ksize)

    # 5. CLAHE + 오츠 이진화
    final = enhance_contrast_and_binarize(blurred,
                                          clip_limit=clahe_clip,
                                          tile_grid_size=clahe_tile)

    # 6. 저장
    safe_imsave(output_path, final)
    return final


### 미학습 PaddleOCR (추론만)

In [20]:
import csv
import subprocess
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image

In [21]:
# GPU/CPU 자동 감지
def detect_gpu():
    try:
        output = subprocess.check_output("nvidia-smi", shell=True, stderr=subprocess.STDOUT)
        return True
    except subprocess.CalledProcessError:
        return False

USE_GPU = detect_gpu()
print(f"[INFO] GPU 사용 여부: {USE_GPU}")

[INFO] GPU 사용 여부: False


In [17]:
# 폰트 다운로드 (한글 폰트)
!wget -q https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Regular.ttf -O /content/NanumGothic.ttf
FONT_PATH = "/content/NanumGothic.ttf"

In [22]:
ocr = PaddleOCR(
    use_angle_cls=True,
    lang='korean',
    rec=True,
    det=True,
    use_gpu=USE_GPU
)

[2025/08/12 11:38:30] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

In [23]:
def run_inference_and_visualize(img_path, save_dir):
    img_path = Path(img_path)
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    # 이미지 로드
    image = safe_imread(img_path)
    if image is None:
        raise FileNotFoundError(f"이미지 로드 실패: {img_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # PaddleOCR 추론
    result = ocr.ocr(str(img_path), cls=True)

    # 결과 파싱
    boxes, txts, scores = [], [], []
    for line in result[0]:
        box, (txt, score) = line
        boxes.append(box)
        txts.append(txt)
        scores.append(score)

    # 폴리곤 시각화
    im_show = draw_ocr(image_rgb, boxes, txts, scores, font_path=FONT_PATH)
    im_show = Image.fromarray(im_show)

    # 결과 이미지 저장
    save_img_path = save_dir / f"{img_path.stem}_ocr_result.jpg"
    im_show.save(save_img_path)

    # CSV 저장 (평가지표용)
    save_csv_path = save_dir / f"{img_path.stem}_ocr_result.csv"
    with open(save_csv_path, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(["x1", "y1", "x2", "y2", "x3", "y3", "x4", "y4", "text", "score"])
        for box, txt, score in zip(boxes, txts, scores):
            flat_box = [coord for point in box for coord in point]
            writer.writerow(flat_box + [txt, score])

    print(f"[저장 완료] 이미지: {save_img_path}, CSV: {save_csv_path}")
    return save_img_path, save_csv_path

In [30]:
def run_inference_and_visualize(img_path, output_dir):
    # 폴더 분리
    img_save_dir = Path(output_dir) / "images"
    csv_save_dir = Path(output_dir) / "csv"
    img_save_dir.mkdir(parents=True, exist_ok=True)
    csv_save_dir.mkdir(parents=True, exist_ok=True)

    # 이미지 로드
    image = safe_imread(img_path)
    if image is None:
        raise FileNotFoundError(f"이미지 로드 실패: {img_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # PaddleOCR 추론
    result = ocr.ocr(str(img_path), cls=True)

    # 결과 파싱
    boxes, txts, scores = [], [], []
    for line in result[0]:
        box, (txt, score) = line
        boxes.append(box)
        txts.append(txt)
        scores.append(score)

    # 폴리곤 시각화
    im_show = draw_ocr(image_rgb, boxes, txts, scores, font_path=FONT_PATH)
    im_show = Image.fromarray(im_show)

    # JPG 저장
    save_img_path = img_save_dir / f"{Path(img_path).stem}_ocr_result.jpg"
    im_show.save(save_img_path)

    # CSV 저장
    save_csv_path = csv_save_dir / f"{Path(img_path).stem}_ocr_result.csv"
    with open(save_csv_path, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(["x1", "y1", "x2", "y2", "x3", "y3", "x4", "y4", "text", "score"])
        for box, txt, score in zip(boxes, txts, scores):
            flat_box = [coord for point in box for coord in point]
            writer.writerow(flat_box + [txt, score])

    return save_img_path, save_csv_path

In [31]:
val_dir = Path("/content/data/원천데이터/TS1/val")
output_dir = Path("/content/pretrained_results")

val_images = list(val_dir.glob("*.jpg"))
for img_path in tqdm(val_images, desc="PaddleOCR Pretrained Inference", unit="img"):
    try:
        run_inference_and_visualize(img_path, output_dir)
    except Exception as e:
        print(f"[ERROR] {img_path} 처리 중 오류: {e}")

PaddleOCR Pretrained Inference:   0%|          | 0/42 [00:00<?, ?img/s]

[2025/08/12 11:50:19] ppocr DEBUG: dt_boxes num : 38, elapsed : 0.40316081047058105
[2025/08/12 11:50:20] ppocr DEBUG: cls num  : 38, elapsed : 0.4670588970184326
[2025/08/12 11:50:38] ppocr DEBUG: rec_res num  : 38, elapsed : 18.29444646835327


PaddleOCR Pretrained Inference:   2%|▏         | 1/42 [00:19<13:30, 19.77s/img]

[2025/08/12 11:50:39] ppocr DEBUG: dt_boxes num : 26, elapsed : 0.34058713912963867
[2025/08/12 11:50:39] ppocr DEBUG: cls num  : 26, elapsed : 0.25953054428100586
[2025/08/12 11:50:57] ppocr DEBUG: rec_res num  : 26, elapsed : 17.79706382751465


PaddleOCR Pretrained Inference:   5%|▍         | 2/42 [00:38<12:45, 19.14s/img]

...

PaddleOCR Pretrained Inference:  95%|█████████▌| 40/42 [12:46<00:44, 22.10s/img]

[2025/08/12 12:03:06] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.35507917404174805
[2025/08/12 12:03:06] ppocr DEBUG: cls num  : 14, elapsed : 0.13435673713684082
[2025/08/12 12:03:19] ppocr DEBUG: rec_res num  : 14, elapsed : 13.282071590423584


PaddleOCR Pretrained Inference:  98%|█████████▊| 41/42 [13:00<00:19, 19.67s/img]

[2025/08/12 12:03:21] ppocr DEBUG: dt_boxes num : 28, elapsed : 1.1062588691711426
[2025/08/12 12:03:21] ppocr DEBUG: cls num  : 28, elapsed : 0.5249338150024414
[2025/08/12 12:03:37] ppocr DEBUG: rec_res num  : 28, elapsed : 15.57778263092041


PaddleOCR Pretrained Inference: 100%|██████████| 42/42 [13:18<00:00, 19.01s/img]


### 학습된 PaddleOCR(FT)

In [27]:
IMG_ROOT = Path("/content/data/원천데이터/OCR")
ANN_ROOT = Path("/content/data/라벨링데이터/OCR")
SAVE_ROOT = Path("/content/ocr_label_txt")
SAVE_ROOT.mkdir(parents=True, exist_ok=True)